In [23]:
from sagas.nlu.ruleset_procs import cached_chunks, get_main_domains
from sagas.conf.conf import cf
from sagas.nlu.nlu_tools import vis_tree

sents,lang='i play football', 'en'
domain, domains = get_main_domains(sents, lang, cf.engine(lang))
for ds in domains:
    vis_tree(ds)

root: play(play, verb, 2)
├── nsubj: i(i, pron, 1)
├── obj: football(football, noun, 3)
└── punct: .(., punct, 4)


In [14]:
from sagas.zh.hownet_helper import build_trees, get_word_sense, get_trees
word='football'
sts=get_trees(word)
st=sts[0]
st.roles

{'domain': {'football|足球', 'sport|体育'},
 'modifier': {'round|圆'},
 'sense': {'football'}}

In [16]:
st.inherits

[SenseNode(role='', name='SportTool|运动器材', children=[SenseNode(role='domain', name='football|足球', children=[]), SenseNode(role='domain', name='sport|体育', children=[]), SenseNode(role='modifier', name='round|圆', children=[])])]

In [17]:
st.has_role(domain='sport|体育'), st.cat_of('SportTool|运动器材')

(True, True)

In [19]:
roles=dict(domain='sport|体育')
v='SportTool|运动器材'
any([st.has_role(**roles) for st in sts]), any([st.cat_of(v) for st in sts])

(True, True)

In [35]:
from typing import Text, Any, Dict, List, Union, Optional
from rx import Observable
from dataclasses import dataclass
@dataclass
class sense_cond:
    part: str
    roles: Optional[Dict]
    cat: Optional[str]
    def is_cat(part, cat):
        return sense_cond(part, None, cat)
    def has_roles(part, **roles):
        return sense_cond(part, roles, None)

sense_cond('/obj', roles, v)

sense_cond(part='/obj', roles={'domain': 'sport|体育'}, cat='SportTool|运动器材')

In [33]:
sense_cond.is_cat('/obj', 'SportTool|运动器材')

sense_cond(part='/obj', roles=None, cat='SportTool|运动器材')

In [50]:
sense_cond.has_roles('/obj', domain='sport|体育')

sense_cond(part='/obj', roles={'domain': 'sport|体育'}, cat=None)

In [41]:
from sagas.nlu.pipes import *
source=get_source('i play football', 'en')

roles=dict(domain='sport|体育')
source.pipe(
    filter_path('/obj'),
    ops.map(lambda t: to_obj({'sense': get_trees(t.lemma), **t})),
    ops.filter(lambda t: any([st.has_role(**roles) for st in t.sense])),
    ops.filter(lambda t: any([st.cat_of(v) for st in t.sense])),
    to_token(), 
).subscribe(
    on_next=lambda value: print(value),
    on_error=lambda e: print(e),
    on_completed=lambda: print('done.'),
)

{'word': 'football/football', 'path': '/obj', 'pos': 'noun'}
done.


In [45]:
any([])

False

In [56]:
def filter_sense(data:sense_cond):
    exprs=[]
    if data.roles:
        print(data.roles)
        exprs.append(ops.filter(lambda t: any([st.has_role(**cond.roles) for st in t.sense])))
    if data.cat:
        exprs.append(ops.filter(lambda t: any([st.cat_of(cond.cat) for st in t.sense])))
    return rx.pipe(*exprs)

# roles=dict(domain='sport|体育')
# cond=sense_cond('/obj', roles, cat='SportTool|运动器材')
# cond=sense_cond('/obj', roles, cat=None)
cond=sense_cond.has_roles('/obj', domain='sport|体育')

source.pipe(
    filter_path(cond.part),
    ops.map(lambda t: to_obj({'sense': get_trees(t.lemma), **t})),
    # filter_sense(cond),
#     ops.filter(lambda t: any([st.has_role(**cond.roles) for st in t.sense])),
#     ops.filter(lambda t: any([st.cat_of(cond.cat) for st in t.sense])),
    filter_sense(cond),
    to_token(), 
).subscribe(
    on_next=lambda value: print(value),
    on_error=lambda e: print(e),
    on_completed=lambda: print('done.'),
)

{'domain': 'sport|体育'}
{'word': 'football/football', 'path': '/obj', 'pos': 'noun'}
done.
